In [ ]:
from pynq import Overlay, allocate
from scipy.fftpack import fft, fftshift
from scipy.signal import freqz
import time
import matplotlib.pyplot as plt
import numpy as np

### 加载 Overlay

In [ ]:
overlay = Overlay("lab4.bit")

### 加载数据

In [ ]:
x = np.load("x.npy")
coeff = np.load("coeff.npy")
fs = 44100
N = len(x)
print("数据长度: %d"%N)
print("采样率: %d Hz"%fs)

In [ ]:
x_fft = fft(x)
x_fft_abs = np.array(np.abs(x_fft)/N*2)[0:N//2]
freqs, resp = freqz(coeff, 1)
freq = (fs/N) * np.array(range(0, N//2))
freq_coeff = np.linspace(0, fs/2.0, len(np.abs(resp)))
plt.figure(figsize=(15, 5))
plt.plot(freq, x_fft_abs/x_fft_abs.max())
plt.plot(freq_coeff, abs(resp/resp.max()))

### 分配内存

In [ ]:
sample_len = len(x)
x_buffer = allocate(shape=(sample_len,), dtype=np.int32)
y_buffer = allocate(shape=(sample_len,), dtype=np.int32)
coef_buffer = allocate(shape=(99,), dtype=np.int32)

In [ ]:
np.copyto(x_buffer, x)
np.copyto(coef_buffer, coeff)

### 配置IP

In [ ]:
fir = overlay.fir_top_0
fir.write(0x1c, x_buffer.physical_address)
fir.write(0x10, y_buffer.physical_address)
fir.write(0x28, coef_buffer.physical_address)
fir.write(0x34, sample_len)

### 启动IP

In [ ]:
fir.write(0x00, 0x01)
ap_done = 0
while(ap_done == 0):
    ap_done = fir.read(0x00) >> 1 & 0x1

In [ ]:
y_fft = fft(y_buffer)
y_fft_abs = np.array(np.abs(y_fft)/N*2)[0:N//2]
plt.figure(figsize=(15, 5))
plt.plot(freq, y_fft_abs/y_fft_abs.max())

### 速度测试

In [ ]:
start_time = time.time()
for i in range(100):
    ap_done = 0
    fir.write(0x00, 0x01)
    while(ap_done == 0):
        ap_done = fir.read(0x00) >> 1 & 0x1
end_time = time.time()
print("平均延迟: %f ms"%((end_time-start_time)*1000/100))